 # Check PyTorch and GPU, clone repo and install dependencies

In [1]:
import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 2.2.1+cu121 (Tesla T4)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
yolotracking_zip_path = "/content/drive/MyDrive/ocean_ai/yolo_tracking-10.0.46"

In [9]:
!unzip $yolotracking_zip_path -d /content

Archive:  /content/drive/MyDrive/ocean_ai/yolo_tracking-10.0.46.zip
72c28b9b497a4d2112d534a1d12b5ec56ccff81d
   creating: /content/yolo_tracking-10.0.46/
   creating: /content/yolo_tracking-10.0.46/.github/
   creating: /content/yolo_tracking-10.0.46/.github/ISSUE_TEMPLATE/
  inflating: /content/yolo_tracking-10.0.46/.github/ISSUE_TEMPLATE/bug.yml  
  inflating: /content/yolo_tracking-10.0.46/.github/ISSUE_TEMPLATE/enhancement.yml  
  inflating: /content/yolo_tracking-10.0.46/.github/ISSUE_TEMPLATE/question.yml  
  inflating: /content/yolo_tracking-10.0.46/.github/dependabot.yml  
   creating: /content/yolo_tracking-10.0.46/.github/workflows/
  inflating: /content/yolo_tracking-10.0.46/.github/workflows/ci.yml  
  inflating: /content/yolo_tracking-10.0.46/.github/workflows/publish.yml  
  inflating: /content/yolo_tracking-10.0.46/.github/workflows/stale.yml  
  inflating: /content/yolo_tracking-10.0.46/.gitignore  
  inflating: /content/yolo_tracking-10.0.46/.pre-commit-config.yaml  
 

In [11]:
%cd /content/yolo_tracking-10.0.46

/content/yolo_tracking-10.0.46


In [18]:
source_copy_path = "/content/drive/MyDrive/ocean_ai/yolo_tracking/source_copy"

In [14]:
command = "cp " + source_copy_path + "/setup.py /content/yolo_tracking-10.0.46"
!{command}

In [16]:
command = "cp " + source_copy_path + "/requirements.txt /content/yolo_tracking-10.0.46"
!{command}

In [17]:
!pip install -e .[yolo]

Obtaining file:///content/yolo_tracking-10.0.46
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.2/204.2 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 91.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 723.1/723.1 kB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.9/98.9 k

# Tracking

환경 설정

In [19]:
command = "cp " + source_copy_path + "/yolov5_2023.pt /content/yolo_tracking-10.0.46"
!{command}

In [21]:
command = "cp " + source_copy_path + "/__init__.py /content/yolo_tracking-10.0.46/examples/detectors"
!{command}

In [22]:
command = "cp " + source_copy_path + "/yolov5.py /content/yolo_tracking-10.0.46/examples/detectors"
!{command}

In [23]:
command = "cp " + source_copy_path + "/ops.py /content/yolo_tracking-10.0.46/boxmot/utils"
!{command}

Tracking 수행

In [23]:
import os

sequenceList = ['L_019_EO_20230925_11.avi', 'L_085_EO_20230925_8.avi', 'L_170_EO_20231104_3.avi']
classList = [[0], [0, 2, 8], [0, 1]] #각 sequence별 추적할 객체 클래스
modelList = ['bytetrack','ocsort', 'strongsort'] #Tracking 알고리즘
track_folder = "/content/yolo_tracking/runs/track"
to_folder = "/content/drive/MyDrive/ocean_ai/yolo_tracking/data/video"#추적 동영상 위치

for i, sequence in enumerate(sequenceList):
  className = classList[i]
  className = ' '.join(map(str, className))
  for modelName in modelList:
    sourcePath = os.path.join(to_folder, sequence)

    !pip uninstall -y ultralytics
    !python examples/track.py --yolo-model yolov5_2023.pt --reid-model lmbn_n_market.pt --tracking-method $modelName --source $sourcePath --save --conf 0.67 --classes $className

    # track 폴더 내의 모든 exp 폴더를 가져옵니다.
    exp_folders = [folder for folder in os.listdir(track_folder) if os.path.isdir(os.path.join(track_folder, folder)) and folder.startswith("exp")]

    # exp 폴더가 없을 경우, 1부터 시작합니다.
    if not exp_folders:
        new_exp_folder_name = "exp"
    else:
        # 가장 큰 숫자를 가진 exp 폴더를 찾습니다.
        max_exp_number = max([int(folder[3:]) for folder in exp_folders])
        # 다음 순번의 exp 폴더 이름을 만듭니다.
        new_exp_folder_name = f"exp{max_exp_number + 1}"

    fromfileName = os.path.join(track_folder, new_exp_folder_name, sequence)
    tofileName = os.path.join(to_folder, sequence.split('.')[0], "_", modelName, ".mp4")

    %cp fromfileName tofileName

Found existing installation: ultralytics 8.1.40
Uninstalling ultralytics-8.1.40:
  Successfully uninstalled ultralytics-8.1.40
2024-04-01 10:42:07.614 | ERROR    | boxmot.utils.checks:check_packages:26 - The 'ultralytics@ git+https://github.com/mikel-brostrom/ultralytics.git' distribution was not found and is required by the application
2024-04-01 10:42:07.614 | WARNING  | boxmot.utils.checks:check_packages:29 - 
Missing packages: "ultralytics @ git+https://github.com/mikel-brostrom/ultralytics.git" 
Atempting installation...
2024-04-01 10:42:15.270 | SUCCESS  | boxmot.utils.checks:check_packages:35 - All the missing packages were installed successfully
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
requirements: Ultralytics requirement ['ultralytics>=8.0.232'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 744.7/744.7 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: ultralytics
    Found exist

FileNotFoundError: [Errno 2] No such file or directory: '/content/yolo_tracking/runs/track'

# Evaluate

환경 설정

In [24]:
val_tools_path = "/content/yolo_tracking-10.0.46/examples/val_utils"

In [25]:
def download_mot_eval_tools(val_tools_path):
    """Download officail evaluation tools for MOT metrics
    Args:
        val_tools_path (pathlib.Path): path to the val tool folder destination
    Returns:
        None
    """
    import git
    from git import Repo
    import subprocess
    val_tools_url = "https://github.com/JonathonLuiten/TrackEval"
    try:
        Repo.clone_from(val_tools_url, val_tools_path)
        print('Official MOT evaluation repo downloaded')
    except git.exc.GitError as err:
        print(f'Eval repo already downloaded {err}')

    # fix deprecated np.float, np.int & np.bool
    cmd_float = f"grep -rl np.float {val_tools_path} | xargs sed -i 's/np.float/float/g'"
    subprocess.run(cmd_float, shell=True)
    cmd_int = f"grep -rl np.int {val_tools_path} | xargs sed -i 's/np.int/int/g'"
    subprocess.run(cmd_int, shell=True)
    cmd_bool = f"grep -rl np.bool {val_tools_path} | xargs sed -i 's/np.bool/bool/g'"
    subprocess.run(cmd_bool, shell=True)

download_mot_eval_tools(val_tools_path)

Official MOT evaluation repo downloaded


In [26]:
command = "cp " + source_copy_path + "/mot_challenge_2d_box_240401.py /content/yolo_tracking-10.0.46/examples/val_utils/trackeval/datasets/mot_challenge_2d_box.py"
!{command}

In [27]:
command = "cp " + source_copy_path + "/run_mot_challenge_240401.py /content/yolo_tracking-10.0.46/examples/val_utils/scripts/run_mot_challenge.py"
!{command}

In [28]:
command = "cp " + source_copy_path + "/val_240401.py /content/yolo_tracking-10.0.46/examples/val.py"
!{command}

In [31]:
command = "cp " + source_copy_path + "/classmap.json /content/yolo_tracking-10.0.46/examples/val_utils"
!{command}

평가 데이터 업로드

In [67]:
%cp -r /content/drive/MyDrive/ocean_ai/yolo_tracking/data/seq/L_019_EO_20230925_11 /content/yolo_tracking-10.0.46/examples/val_utils/data/

In [55]:
%cp -r /content/drive/MyDrive/ocean_ai/yolo_tracking/data/seq/L_085_EO_20230925_8 /content/yolo_tracking-10.0.46/examples/val_utils/data/

In [56]:
%cp -r /content/drive/MyDrive/ocean_ai/yolo_tracking/data/seq/L_170_EO_20231104_3 /content/yolo_tracking-10.0.46/examples/val_utils/data/

평가 수행

In [ ]:
import json
sequenceNameList = ['L_019_EO_20230925_11', 'L_085_EO_20230925_8', 'L_170_EO_20231104_3']
classList = [[0], [0, 2, 8], [0, 1]] #각 sequence별 평가할 객체 클래스
classmapFile = "/content/yolo_tracking-10.0.46/examples/val_utils/classmap.json" #classmap 파일 경로
modelList = ['bytetrack','ocsort', 'strongsort'] #Tracking 알고리즘
val_folder = "/content/yolo_tracking/runs/val"
to_folder = "/content/drive/MyDrive/ocean_ai/" #MOT 결과를 저장할 위치

#classmap 불러오기
with open(classmapFile, 'r') as file:
  classmap = json.load(file)
reverse_classmap = {value: key for key, value in classmap.items()}

for i, sequence in enumerate(sequenceNameList):
  classList_seq = classList[i]
  for class_seq in classList_seq:
    className = reverse_classmap.get(class_seq)
    for modelName in modelList:

      #평가 수행
      !pip uninstall -y ultralytics
      !python examples/val.py --yolo-model yolov5_2023.pt --reid-model lmbn_n_market.pt --tracking-method $modelName --benchmark $sequence --classes $class_seq --imgsz 640 --conf 0.67 --className $className --classMap $classmapFile

      # val 폴더 내의 모든 exp 폴더를 가져옵니다.
      exp_folders = [folder for folder in os.listdir(val_folder) if os.path.isdir(os.path.join(val_folder, folder)) and folder.startswith("exp")]

      # exp 폴더가 없을 경우, 1부터 시작합니다.
      if not exp_folders:
          new_exp_folder_name = "exp"
      else:
          # 가장 큰 숫자를 가진 exp 폴더를 찾습니다.
          max_exp_number = max([int(folder[3:]) for folder in exp_folders])
          # 다음 순번의 exp 폴더 이름을 만듭니다.
          new_exp_folder_name = f"exp{max_exp_number + 1}"

      fromfileName = os.path.join(val_folder, new_exp_folder_name, "MOT_results.txt")
      tofileName = os.path.join(to_folder, sequence, "_MOT_results_", modelName, ".txt")

      %cp fromfileName tofileName